In [1]:
#Install tabula python wrapper
!pip install tabula-py

In [0]:
from tabula import read_pdf
import pandas as pd
from datetime import date

In [0]:
# Use the url from www.gob.mx
# TODO: check how their cms works, seems like every day they upload 4 pdfs... 
# check file/<number>/ and date at the end

sospechosos ="https://www.gob.mx/cms/uploads/attachment/file/542353/Tabla_casos_sospechosos_COVID-19_2020.03.19.pdf"

In [0]:
# specify the area for page 1 since the column names are not being picked up by tabula
# A primer on PDF coordinate system to pass values to area param
# https://www.leadtools.com/help/leadtools/v19/dh/to/pdf-topics-pdfcoordinatesystem.html
df = read_pdf(sospechosos, area=[99, 0, 792, 792], pages="1")

In [0]:
# Get the rest of the pages in a second data frame 
# TODO: seems like is not reading the first row, maybe because it doesnt have the upper border
df2 = read_pdf(sospechosos, pages="all")

# Cleaning the first data frame

In [6]:
# Tabula outputs unformatted headers...
df[0].head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,1,SAN LUIS POTOSÍ,F,22,11/03/2020,Sospechoso,España,06/03/2020,NaN
1,2,CIUDAD DE MÉXICO,F,26,13/03/2020,Sospechoso,España,13/03/2020,NaN
2,3,MÉXICO,M,23,03/03/2020,Sospechoso,España,02/03/2020,NaN
3,4,MÉXICO,M,34,16/02/2020,Sospechoso,Italia,29/02/2020,NaN
4,5,CIUDAD DE MÉXICO,F,30,07/03/2020,Sospechoso,España,28/02/2020,NaN


In [0]:
# drop the first row with non usable data
df0 = df[0]

# Rename the columns
newcols = {
    'Unnamed: 0': 'No de caso', 
    'Unnamed: 1': 'Estado', 
    'Unnamed: 2': 'Sexo',
    'Unnamed: 3': 'Edad',
    'Unnamed: 4': 'Fecha de Inicio de síntomas',
    'Unnamed: 5': 'Identificación de COVID-19 por RTPCR en tiempo real',
    'Unnamed: 6': 'Procedencia',
    'Unnamed: 7': 'Fecha de llegada a México'
}

# Throws a warning in colab, unfortunatly I'm still not proficient with python...
df0.rename(columns=newcols, inplace=True)

In [8]:
df0.head()

,No de caso,Estado,Sexo,Edad,Fecha de Inicio de síntomas,Identificación de COVID-19 por RTPCR en tiempo real,Procedencia,Fecha de llegada a México,Unnamed: 8
0,1,SAN LUIS POTOSÍ,F,22,11/03/2020,Sospechoso,España,06/03/2020,NaN
1,2,CIUDAD DE MÉXICO,F,26,13/03/2020,Sospechoso,España,13/03/2020,NaN
2,3,MÉXICO,M,23,03/03/2020,Sospechoso,España,02/03/2020,NaN
3,4,MÉXICO,M,34,16/02/2020,Sospechoso,Italia,29/02/2020,NaN
4,5,CIUDAD DE MÉXICO,F,30,07/03/2020,Sospechoso,España,28/02/2020,NaN


In [0]:
# Remove the extra columns:
columns = ['Unnamed: 8']
df0 = df0.drop(labels=columns, axis=1)

In [10]:
df0.head()

,No de caso,Estado,Sexo,Edad,Fecha de Inicio de síntomas,Identificación de COVID-19 por RTPCR en tiempo real,Procedencia,Fecha de llegada a México
0,1,SAN LUIS POTOSÍ,F,22,11/03/2020,Sospechoso,España,06/03/2020
1,2,CIUDAD DE MÉXICO,F,26,13/03/2020,Sospechoso,España,13/03/2020
2,3,MÉXICO,M,23,03/03/2020,Sospechoso,España,02/03/2020
3,4,MÉXICO,M,34,16/02/2020,Sospechoso,Italia,29/02/2020
4,5,CIUDAD DE MÉXICO,F,30,07/03/2020,Sospechoso,España,28/02/2020


## Concatenate with the other data frames

In [11]:
# Data frame 2 should have all pages...
len(df2)

9

In [12]:
## Remove page 1 since we already have it
df2 = df2[1:]
df2[0].head()

,46,CIUDAD DE MÉXICO,F,44,15/03/2020,Sospechoso,Francia,15/03/2020.1
0,47,QUINTANA ROO,M,45,15/03/2020,Sospechoso,Contacto,15/03/2020
1,48,JALISCO,M,54,13/03/2020,Sospechoso,Contacto,NA
2,49,VERACRUZ,F,34,12/03/2020,Sospechoso,Italia,14/03/2020
3,50,CIUDAD DE MÉXICO,F,22,11/03/2020,Sospechoso,España,08/03/2020
4,51,HIDALGO,F,23,14/03/2020,Sospechoso,Alemania,15/03/2020


In [0]:
# Add column headers to data frames
cols = ['No de caso', 
    'Estado', 
    'Sexo',
    'Edad',
    'Fecha de Inicio de síntomas',
    'Identificación de COVID-19 por RTPCR en tiempo real',
    'Procedencia',
    'Fecha de llegada a México'
]

# Initialize list with the cleaned df 0 
dataFrames = [df0]

# Iterate through the dataframes in df2 and add the column headers
for dfx in df2:
    # Add header value as data
    first = dfx.columns
    first_row = dict( zip(cols, first))
    data = pd.DataFrame([first_row])
    dataframe = pd.concat([data, pd.DataFrame(dfx.values, columns=cols)])
    # append the dataframes
    dataFrames.append(dataframe)


In [14]:
# Concatenate the cleaned data frames
data = pd.concat(dataFrames,ignore_index=True)
data

,No de caso,Estado,Sexo,Edad,Fecha de Inicio de síntomas,Identificación de COVID-19 por RTPCR en tiempo real,Procedencia,Fecha de llegada a México
0,1,SAN LUIS POTOSÍ,F,22,11/03/2020,Sospechoso,España,06/03/2020
1,2,CIUDAD DE MÉXICO,F,26,13/03/2020,Sospechoso,España,13/03/2020
2,3,MÉXICO,M,23,03/03/2020,Sospechoso,España,02/03/2020
3,4,MÉXICO,M,34,16/02/2020,Sospechoso,Italia,29/02/2020
4,5,CIUDAD DE MÉXICO,F,30,07/03/2020,Sospechoso,España,28/02/2020
...,...,...,...,...,...,...,...,...
435,444,CIUDAD DE MÉXICO,F,57,15/03/2020,Sospechoso,Estados Unidos,11/03/2020
436,445,MÉXICO,M,33,10/03/2020,Sospechoso,Estados Unidos,10/03/2020
437,446,CIUDAD DE MÉXICO,M,40,09/03/2020,Sospechoso,Italia,08/03/2020
438,447,MÉXICO,M,15,08/03/2020,Sospechoso,Estados Unidos,08/03/2020


## Export the data set:

In [0]:
today = date.today()
d1 = today.strftime("%Y-%m-%d")
filename = "sospechosos-"+ d1+".csv"
data.to_csv(filename)